In [2]:
import json
import pandas as pd
import time
from tqdm import tqdm
import re
import datetime

In [3]:
#获取202207往前12个月的月份list（为了后续读取文件）
month_date_list = list(pd.period_range('2022-07',periods=12,freq='M'))
monthlist = list(map(lambda x : str(x)[:4] + str(x)[-2:] , month_date_list))

In [4]:
#读取全部热搜文件并合并
monthallnewslist = []
for month in tqdm(monthlist):
    news = pd.read_excel(month + '.xlsx')
    monthallnewslist.append(news.drop('序号',axis=1))
newsall = pd.concat(monthallnewslist)
newsall.sort_values('上榜时间',inplace=True)
newsalldf = newsall.set_index('上榜时间')
newsalldf.index = pd.to_datetime(newsalldf.index)

100%|██████████| 12/12 [00:10<00:00,  1.10it/s]


In [5]:
#获取交易日
close = pd.read_feather('BasicFactor_Close.txt').set_index('time')
close.index = pd.to_datetime(close.index.astype(str))
tradeday = close['2022-07-01':].index

In [78]:
#构造交易日开盘时间，交易日停盘时间，前一个交易日开盘时间
start_tradeday = list(map(lambda x : x + datetime.timedelta(hours=9,minutes=15) , tradeday))
end_tradeday = list(map(lambda x : x + datetime.timedelta(hours=15) , tradeday))
previous_end_tradeday = end_tradeday.copy()
previous_end_tradeday.insert(0,pd.Timestamp('2022-06-30 15:00:00'))
previous_end_tradeday.pop()

Timestamp('2023-06-30 15:00:00')

In [79]:
#盘前news；盘中news，盘后news
preopen_timepair_list = list(zip(previous_end_tradeday,start_tradeday))
opening_timepair_list = list(zip(start_tradeday,end_tradeday))
all_timepair_list = list(zip(previous_end_tradeday,end_tradeday))

In [80]:
#获取三组热搜数据表
def getnewscut(newsdf,timepairlist):
    daynewslist = []
    for timepair in timepairlist:
        daynews = newsdf.truncate(before=timepair[0], after=timepair[1])
        day = timepair[1].date()
        daynews = pd.concat([daynews,pd.Series(day,name='date',index=daynews.index)],axis=1)
        daynewslist.append(daynews)
    finalnews = pd.concat(daynewslist)
    return finalnews

In [81]:
preopen_news = getnewscut(newsalldf,preopen_timepair_list)
opening_news = getnewscut(newsalldf,opening_timepair_list)
all_news = getnewscut(newsalldf,all_timepair_list)

In [82]:
preopen_news.to_csv('preopen_news.csv')
opening_news.to_csv('opening_news.csv')
all_news.to_csv('all_news.csv')